In [1]:
 #!pip install librosa


In [2]:
"""import librosa as lib
import os
import pandas as pd
import pylab
import numpy as np
import librosa.display
import glob """

In [3]:
"""def convert(filename):   
    sig, fs = lib.load(filename)   
    # make pictures name 
    save_path = filename+'.jpg'

    pylab.axis('off') # no axis
    pylab.axes([0., 0., 1., 1.], frameon=False, xticks=[], yticks=[]) # Remove the white edge
    S = librosa.feature.melspectrogram(y=sig, sr=fs)
    lib.display.specshow(librosa.power_to_db(S, ref=np.max))
    pylab.savefig(save_path, bbox_inches=None, pad_inches=0)
    pylab.close()"""

In [4]:
##"""for i in range(11):
#    for filename in glob.glob(r'D:\Projects\urban_sound\UrbanSound8K\audio\fold'+str(i)+'\*.wav'):
 #       convert(filename)"""

In [1]:
import pandas as pd
data = pd.read_csv(r'D:\Projects\urban_sound\UrbanSound8K\metadata\UrbanSound8K.csv')

In [2]:
print(data.head())

      slice_file_name    fsID  start        end  salience  fold  classID  \
0    100032-3-0-0.wav  100032    0.0   0.317551         1     5        3   
1  100263-2-0-117.wav  100263   58.5  62.500000         1     5        2   
2  100263-2-0-121.wav  100263   60.5  64.500000         1     5        2   
3  100263-2-0-126.wav  100263   63.0  67.000000         1     5        2   
4  100263-2-0-137.wav  100263   68.5  72.500000         1     5        2   

              class  
0          dog_bark  
1  children_playing  
2  children_playing  
3  children_playing  
4  children_playing  


In [3]:
test = data.loc[data['fold'] == 10]
train = data.loc[data['fold'] != 10]

In [4]:
#cv2.imshow('in',cv2.imread(r'D:\Projects\urban_sound\UrbanSound8K\audio\fold7\99812-1-6-0.wav.jpg'))


In [5]:
from sklearn.preprocessing import LabelEncoder
import numpy as np
import os
import cv2

loc = r'D:\Projects\urban_sound\UrbanSound8K\audio'
img_lst = np.array(train.slice_file_name.tolist())
folds = np.array(train.fold)
print(img_lst.shape)
img_lst = [loc +'\\'+'fold'+str(fold)+'\\'+ s + '.jpg' for s,fold in zip(img_lst,folds)]
X = []
for s in img_lst:
    X.append(cv2.resize(cv2.imread(s, cv2.IMREAD_COLOR),(331,331),interpolation=cv2.INTER_CUBIC))
#X = [cv2.imshow('img',s) for s in X]

y = np.array(train.classID.tolist())

(7895,)


In [6]:
import gc
del train
gc.collect()

0

In [7]:
X = np.array(X)
print(X[0])
y

[[[  5   0   0]
  [  5   0   0]
  [  5   0   0]
  ...
  [  4   0   0]
  [  4   0   0]
  [  4   0   0]]

 [[  4   0   0]
  [  4   0   0]
  [  4   0   0]
  ...
  [  4   0   0]
  [  4   0   0]
  [  4   0   0]]

 [[  3   0   0]
  [  3   0   0]
  [  3   0   0]
  ...
  [  4   0   0]
  [  4   0   0]
  [  4   0   0]]

 ...

 [[105  14  55]
  [105  14  55]
  [105  14  55]
  ...
  [ 81  19  35]
  [ 81  19  35]
  [ 81  19  35]]

 [[ 91  14  43]
  [ 91  14  43]
  [ 91  14  43]
  ...
  [ 62  13  25]
  [ 62  13  25]
  [ 62  13  25]]

 [[ 87  18  38]
  [ 87  18  38]
  [ 87  18  38]
  ...
  [ 57  15  26]
  [ 57  15  26]
  [ 57  15  26]]]


array([3, 2, 2, ..., 1, 1, 1])

In [8]:
X = X.astype('float32')

In [9]:
X /= 255 

In [10]:
# construct the image generator for data augmentation
from keras.preprocessing.image import ImageDataGenerator
aug = ImageDataGenerator()

Using TensorFlow backend.


In [11]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
lb = LabelEncoder()
y = np_utils.to_categorical(lb.fit_transform(y))

In [12]:
import keras
base = keras.applications.nasnet.NASNetLarge(input_shape=None, include_top=False, weights=None, input_tensor=None, pooling=None, classes=10)

In [15]:
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.optimizers import SGD

BS = 12
EPOCHS = 50
x = base.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)
model = Model(inputs=base.input, outputs=predictions)

for layer in base.layers:
    layer.trainable = False
model.compile(optimizer=SGD(lr=0.01, momentum=0.9), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.fit_generator(aug.flow(X, y, batch_size=BS),
	steps_per_epoch=len(X) // BS,
	epochs=EPOCHS, verbose=1)
model.save(args["model"])

NameError: name 'trainX' is not defined